In [59]:
!pip install tabulate

In [1]:
import re

# Defining functions

In [2]:
def is_relop(token):
    if re.search("==|>=|<=|<>|<|>", token) is not None:
        if token == re.search("==|>=|<=|<>|<|>", token).group():
            return 1;
        else:
            return 0;
    else:
        return 0;

In [3]:
def is_digit(token):
    if re.search("[+-]?\d+[.]\d+|[+-]?\d+", token) is not None:
        if token == re.search("[+-]?\d+[.]\d+|[+-]?\d+", token).group():
            return 1;
        elif token == re.search("[+-]?\d+[.]\d+|[+-]?\d+", token).group() + ';':
            return 1;
        else:
            return 0;
    else:
        return 0;

In [4]:
def is_keyword(token):
    TYPE = 'int'
    MAIN = "main"
    BEGIN = 'begin'
    DO = 'do'; WHILE = 'while'
    RETURN = 'return'
    END = 'end'
    keywords = [TYPE, MAIN, BEGIN, DO, WHILE, RETURN, END]
    if token in keywords:
        if token==TYPE:
            return 'TYPE'
        else:
            return 1
    elif re.search("^while[(].*[)]", token) is not None:
        return 1
    else:
        return 0

In [5]:
def is_identifier(token, ids, pos):
    if re.search("[a-zA-Z][a-zA-Z0-9]*", token) is not None:
        if is_keyword(token):
            return 0
        if token == re.search("[a-zA-Z][a-zA-Z0-9]*", token).group():
            return 1
        else:
            return 0;
    else:
        return 0;

In [6]:
def is_whitespace(token):
    if re.search("\s+", token) is not None:
        if token == re.search("\s+", token).group():
            return 1;
        else:
            return 0;
    else:
        return 0;

# Taking input

#### Grammar

S: FORMAT \
FORMAT: TYPE MAIN BEGIN CONTENT END \
TYPE: 'int' | 'float' \
CONTENT: TYPE id ';' DOWHILE RETURN \
DOWHILE: 'do' OPERATION 'while' '(' EXP ')' \
OPERATION: id '=' VAR '+' VAR ';' id '=' VAR ';' \
EXP: 'exp' | id RELOP id \
RETURN: 'return' '(' VAR ')' \
VAR: id | num

In [7]:
with open('code.txt', 'r') as f:
    content = f.read()
print(content)

int main()
begin
int n;
do
    expr = expr + expr;
    n = blahh;
    y = mx + c;
while(a == b)
return(n)
end


# Preprocessing

In [8]:
code_lex = ''

for i in content:
    if i=='(':
        code_lex += ' ( '
    elif i==')':
        code_lex += ' ) '
    elif i==';':
        code_lex += ' ;'
    else:
        code_lex += i

In [9]:
print(code_lex)

int main (  ) 
begin
int n ;
do
    expr = expr + expr ;
    n = blahh ;
    y = mx + c ;
while ( a == b ) 
return ( n ) 
end


In [10]:
tokens = code_lex.split()
print(tokens)

['int', 'main', '(', ')', 'begin', 'int', 'n', ';', 'do', 'expr', '=', 'expr', '+', 'expr', ';', 'n', '=', 'blahh', ';', 'y', '=', 'mx', '+', 'c', ';', 'while', '(', 'a', '==', 'b', ')', 'return', '(', 'n', ')', 'end']


In [11]:
symbol_table = []
ids = {}

In [12]:
pos = 0
for token in tokens:
    # token, type, location 
    if is_keyword(token)=='TYPE':
        symbol_table.append([token,'TYPE',''])
    elif is_keyword(token)==1:
        symbol_table.append([token,'KEYWORD',''])
    elif is_relop(token):
        symbol_table.append([token,'RELOP',''])
    elif is_identifier(token, ids, pos)!=0:
        if token in ids.keys():
            pass
        else:
            pos += 1
            ids[token] = pos
        symbol_table.append([token,'IDENTIFIER',ids[token]])
    elif is_digit(token)==1:
        symbol_table.append([token,'NUM',''])
    elif token==';':
        symbol_table.append([token,'SC',''])
    elif token=='=':
        symbol_table.append([token,'ASSIGNMENT',''])
    elif token=='(':
        symbol_table.append([token,'BRACKET_OPEN',''])
    elif token==')':
        symbol_table.append([token,'BRACKET_CLOSE',''])
print(symbol_table)

[['int', 'TYPE', ''], ['main', 'KEYWORD', ''], ['(', 'BRACKET_OPEN', ''], [')', 'BRACKET_CLOSE', ''], ['begin', 'KEYWORD', ''], ['int', 'TYPE', ''], ['n', 'IDENTIFIER', 1], [';', 'SC', ''], ['do', 'KEYWORD', ''], ['expr', 'IDENTIFIER', 2], ['=', 'ASSIGNMENT', ''], ['expr', 'IDENTIFIER', 2], ['expr', 'IDENTIFIER', 2], [';', 'SC', ''], ['n', 'IDENTIFIER', 1], ['=', 'ASSIGNMENT', ''], ['blahh', 'IDENTIFIER', 3], [';', 'SC', ''], ['y', 'IDENTIFIER', 4], ['=', 'ASSIGNMENT', ''], ['mx', 'IDENTIFIER', 5], ['c', 'IDENTIFIER', 6], [';', 'SC', ''], ['while', 'KEYWORD', ''], ['(', 'BRACKET_OPEN', ''], ['a', 'IDENTIFIER', 7], ['==', 'RELOP', ''], ['b', 'IDENTIFIER', 8], [')', 'BRACKET_CLOSE', ''], ['return', 'KEYWORD', ''], ['(', 'BRACKET_OPEN', ''], ['n', 'IDENTIFIER', 1], [')', 'BRACKET_CLOSE', ''], ['end', 'KEYWORD', '']]


In [13]:
print("TOKEN \t\t TYPE  \t\t     LOCATION")
print("---------------------------------------------")
for sym in symbol_table:
    print(sym[0], '\t\t', sym[1], '\t\t', sym[2])

TOKEN 		 TYPE  		     LOCATION
---------------------------------------------
int 		 TYPE 		 
main 		 KEYWORD 		 
( 		 BRACKET_OPEN 		 
) 		 BRACKET_CLOSE 		 
begin 		 KEYWORD 		 
int 		 TYPE 		 
n 		 IDENTIFIER 		 1
; 		 SC 		 
do 		 KEYWORD 		 
expr 		 IDENTIFIER 		 2
= 		 ASSIGNMENT 		 
expr 		 IDENTIFIER 		 2
expr 		 IDENTIFIER 		 2
; 		 SC 		 
n 		 IDENTIFIER 		 1
= 		 ASSIGNMENT 		 
blahh 		 IDENTIFIER 		 3
; 		 SC 		 
y 		 IDENTIFIER 		 4
= 		 ASSIGNMENT 		 
mx 		 IDENTIFIER 		 5
c 		 IDENTIFIER 		 6
; 		 SC 		 
while 		 KEYWORD 		 
( 		 BRACKET_OPEN 		 
a 		 IDENTIFIER 		 7
== 		 RELOP 		 
b 		 IDENTIFIER 		 8
) 		 BRACKET_CLOSE 		 
return 		 KEYWORD 		 
( 		 BRACKET_OPEN 		 
n 		 IDENTIFIER 		 1
) 		 BRACKET_CLOSE 		 
end 		 KEYWORD 		 


In [14]:
# Rules
rules = {
'r0' : "S: FORMAT",
'r1' : "FORMAT: TYPE main ( ) begin CONTENT end",
'r2' : "TYPE: int",
'r3' : "TYPE: float",
'r4' : "CONTENT: TYPE id ; DOWHILE RETURN",
'r5' : "DOWHILE: do OPERATION while ( EXP )",
'r6' : "OPERATION: id = VAR + VAR ; id = VAR ;",
'r7' : "EXP: exp",
'r8' : "EXP: id relop id",
'r9' : "RETURN: return ( VAR )",
'r10' : "VAR: id",
'r11' : "VAR: num",
}

In [15]:
non_terminals = ['TYPE', 'CONTENT', 'DOWHILE', 'RETURN', 'OPERATION',
                'EXP', 'VAR', 'FORMAT']
terminals = ['id', 'num', ';', '(', ')', '=', '+', 'while', 'return', 'exp', 'relop']

In [16]:
augmented_grammar = {
    'S': ['. FORMAT'],
    'FORMAT': [". TYPE main ( ) begin CONTENT end"],
    'TYPE': ['. int', '. float'],
    'CONTENT': [". TYPE id ; DOWHILE RETURN"],
    'DOWHILE': [". do OPERATION while ( EXP )"],
    'OPERATION': [". id = VAR + VAR ;", ". id = VAR ;"],
    'EXP': ['. exp', '. id relop id'],
    'RETURN': ['. return ( VAR )'],
    'VAR': ['. id', '. num']
}

In [17]:
parse_table = []

In [18]:
def find_closure(rule):
  closure = []
  closure.append(rule)

  flag = True
  while(flag):
    flag = False

    for item in closure:
      if item.split(': ')[1].find('.') != -1:
        prod = item.split(': ')[1]
        dot_pos = prod.find('.')
        if re.search(r"[.] [a-zA-Z_();]+", prod) is not None:
            sym_after_dot = re.search(r"[.] [a-zA-Z_();]+", prod).group()[2:]
        else:
            return closure

        if sym_after_dot in non_terminals:
          rule = sym_after_dot

          if rule in augmented_grammar.keys():
            for i in augmented_grammar[rule]:
              new_rule = rule + ': ' + i

              if new_rule not in closure:
                closure.append(new_rule)
                flag = True
  return closure

In [19]:
closure_s = find_closure('S: FORMAT .')

for i in closure_s:
    print(i)

S: FORMAT .


In [20]:
closure_content = find_closure('CONTENT: TYPE id ; . DOWHILE RETURN')

for i in closure_content:
    print(i)

CONTENT: TYPE id ; . DOWHILE RETURN
DOWHILE: . do OPERATION while ( EXP )


In [21]:
closure_s

['S: FORMAT .']

In [22]:
def goto(I,X):
    goto_list = []
    if re.search(r"[.] [a-zA-Z_();]+", I[0]) is None:
        return goto_list
        
    for item in I:
        if re.search("[.] [a-zA-Z_();]+", item).group() != '. '+X:
            continue
        # Shifting the dot
        prod_span = re.search("[.] [a-zA-Z_();]+", item).span()
        item = re.sub(' [.] ', ' ', item)
        newprod = item[:prod_span[1]-2] + ' . ' + item[prod_span[1]-1:]
        if (newprod[-1]=='.') or (newprod[-2:]=='. '):
            closure_items = find_closure(newprod)
            for i in closure_items:
                goto_list.append(i)
            return goto_list
    closure_items = find_closure(newprod)
    for i in closure_items:
        goto_list.append(i)
    return goto_list

In [23]:
action = {}

def items():
    C = []
    C.append(find_closure('S: . FORMAT'))
    k = 0
    
    flag = True
    while flag:
        flag = False

        for I in C:
            for item in I:
                production = item.split(': ')[1]
                head = item.split(': ')[0]
                if re.search(r"[.] [a-zA-Z_();]+", production) is not None:
                    X = re.search(r"[.] [a-zA-Z_();]+", production).group()[2:]
                    gotovals = goto(I,X)
                    if len(goto(I,X))!=0 and gotovals not in C:
                        k += 1
                        C.append(goto(I,X))
                        if X in terminals:
                            lst = ['S']
                            lst.append(k)
                            action[C.index(I)] = {f'{X}': lst}
                        if X in non_terminals:
                            lst = ['']
                            lst.append(k)
                            action[C.index(I)] = {f'{X}': lst}
                        flag = True
    return C, action

In [24]:
C, action = items()

In [25]:
action

{0: {'TYPE': ['', 2]},
 5: {'(': ['S', 6]},
 6: {')': ['S', 7]},
 8: {'TYPE': ['', 10]},
 10: {'id': ['S', 12]},
 12: {';': ['S', 13]},
 13: {'DOWHILE': ['', 14]},
 14: {'return': ['S', 17]},
 15: {'id': ['S', 19]},
 17: {'(': ['S', 20]},
 18: {'while': ['S', 21]},
 20: {'num': ['S', 24]},
 21: {'(': ['S', 25]},
 22: {')': ['S', 26]},
 25: {'id': ['S', 29]},
 27: {')': ['S', 30]},
 29: {'relop': ['S', 31]},
 31: {'id': ['S', 32]}}

In [26]:
C

[['S: . FORMAT',
  'FORMAT: . TYPE main ( ) begin CONTENT end',
  'TYPE: . int',
  'TYPE: . float'],
 ['S: FORMAT . '],
 ['FORMAT: TYPE . main ( ) begin CONTENT end'],
 ['TYPE: int . '],
 ['TYPE: float . '],
 ['FORMAT: TYPE main . ( ) begin CONTENT end'],
 ['FORMAT: TYPE main ( . ) begin CONTENT end'],
 ['FORMAT: TYPE main ( ) . begin CONTENT end'],
 ['FORMAT: TYPE main ( ) begin . CONTENT end',
  'CONTENT: . TYPE id ; DOWHILE RETURN',
  'TYPE: . int',
  'TYPE: . float'],
 ['FORMAT: TYPE main ( ) begin CONTENT . end'],
 ['CONTENT: TYPE . id ; DOWHILE RETURN'],
 ['FORMAT: TYPE main ( ) begin CONTENT end . '],
 ['CONTENT: TYPE id . ; DOWHILE RETURN'],
 ['CONTENT: TYPE id ; . DOWHILE RETURN',
  'DOWHILE: . do OPERATION while ( EXP )'],
 ['CONTENT: TYPE id ; DOWHILE . RETURN', 'RETURN: . return ( VAR )'],
 ['DOWHILE: do . OPERATION while ( EXP )',
  'OPERATION: . id = VAR + VAR ;',
  'OPERATION: . id = VAR ;'],
 ['CONTENT: TYPE id ; DOWHILE RETURN . '],
 ['RETURN: return . ( VAR )'],
 ['DO

In [27]:
augmented_grammar = {
    'S': ['. FORMAT'],
    'FORMAT': [". TYPE main ( ) begin CONTENT end"],
    'TYPE': ['. int', '. float'],
    'CONTENT': [". TYPE id ; DOWHILE RETURN"],
    'DOWHILE': [". do OPERATION while ( EXP )"],
    'OPERATION': [". id = VAR + VAR ;", ". id = VAR ;"],
    'EXP': ['. exp', '. id relop id'],
    'RETURN': ['. return ( VAR )'],
    'VAR': ['. id', '. num']
}

In [28]:
follow = {
    'FORMAT': ['$'],
    'TYPE': ['main','id'],
    'CONTENT': ['end'],
    'DOWHILE': ['return'],
    'OPERATION': ['while'],
    'EXP': [')'],
    'RETURN': ['end'],
    'VAR': [';','=','relop','+']
}

In [29]:
action

{0: {'TYPE': ['', 2]},
 5: {'(': ['S', 6]},
 6: {')': ['S', 7]},
 8: {'TYPE': ['', 10]},
 10: {'id': ['S', 12]},
 12: {';': ['S', 13]},
 13: {'DOWHILE': ['', 14]},
 14: {'return': ['S', 17]},
 15: {'id': ['S', 19]},
 17: {'(': ['S', 20]},
 18: {'while': ['S', 21]},
 20: {'num': ['S', 24]},
 21: {'(': ['S', 25]},
 22: {')': ['S', 26]},
 25: {'id': ['S', 29]},
 27: {')': ['S', 30]},
 29: {'relop': ['S', 31]},
 31: {'id': ['S', 32]}}

In [30]:
for c in C:
    for item in c:
        print(c, C.index(c))
        if item.find('. ') - len(item) == -2:
            if item == 'S: FORMAT . ':
                try:
                    action[C.index(c)]['$'] = 'acc'
                except:
                    action[C.index(c)] = {}
                    action[C.index(c)]['$'] = 'acc'
                # print(action)
            else:
                prod = item.split(': ')[0]
                for f in follow[prod]:
                    org_prod = prod + ": " + item.split(': ')[1][:-2]
                    # print(org_prod)
                    try:
                        print(C.index(c),f)
                        action[int(C.index(c))][f] = ['R']
                        action[int(C.index(c))][f].append(key[1:])
                        # print(f)
                    except KeyError:
                        action[C.index(c)] = {}
                        action[int(C.index(c))][f] = ['R']
                        for key, val in rules.items():
                            # print(key, val,org_prod)
                            if val == org_prod:
                                action[int(C.index(c))][f].append(key[1:])
                        # action[C.index(c)][f] = []
                    

['S: . FORMAT', 'FORMAT: . TYPE main ( ) begin CONTENT end', 'TYPE: . int', 'TYPE: . float'] 0
['S: . FORMAT', 'FORMAT: . TYPE main ( ) begin CONTENT end', 'TYPE: . int', 'TYPE: . float'] 0
['S: . FORMAT', 'FORMAT: . TYPE main ( ) begin CONTENT end', 'TYPE: . int', 'TYPE: . float'] 0
['S: . FORMAT', 'FORMAT: . TYPE main ( ) begin CONTENT end', 'TYPE: . int', 'TYPE: . float'] 0
['S: FORMAT . '] 1
['FORMAT: TYPE . main ( ) begin CONTENT end'] 2
['TYPE: int . '] 3
3 main
3 id
['TYPE: float . '] 4
4 main
4 id
['FORMAT: TYPE main . ( ) begin CONTENT end'] 5
['FORMAT: TYPE main ( . ) begin CONTENT end'] 6
['FORMAT: TYPE main ( ) . begin CONTENT end'] 7
['FORMAT: TYPE main ( ) begin . CONTENT end', 'CONTENT: . TYPE id ; DOWHILE RETURN', 'TYPE: . int', 'TYPE: . float'] 8
['FORMAT: TYPE main ( ) begin . CONTENT end', 'CONTENT: . TYPE id ; DOWHILE RETURN', 'TYPE: . int', 'TYPE: . float'] 8
['FORMAT: TYPE main ( ) begin . CONTENT end', 'CONTENT: . TYPE id ; DOWHILE RETURN', 'TYPE: . int', 'TYPE: 

In [31]:
action

{0: {'TYPE': ['', 2]},
 5: {'(': ['S', 6]},
 6: {')': ['S', 7]},
 8: {'TYPE': ['', 10]},
 10: {'id': ['S', 12]},
 12: {';': ['S', 13]},
 13: {'DOWHILE': ['', 14]},
 14: {'return': ['S', 17]},
 15: {'id': ['S', 19]},
 17: {'(': ['S', 20]},
 18: {'while': ['S', 21]},
 20: {'num': ['S', 24]},
 21: {'(': ['S', 25]},
 22: {')': ['S', 26]},
 25: {'id': ['S', 29]},
 27: {')': ['S', 30]},
 29: {'relop': ['S', 31]},
 31: {'id': ['S', 32]},
 1: {'$': 'acc'},
 3: {'main': ['R'], 'id': ['R', '11']},
 4: {'main': ['R'], 'id': ['R', '11']},
 11: {'$': ['R']},
 16: {'end': ['R']},
 23: {';': ['R'], '=': ['R', '11'], 'relop': ['R', '11'], '+': ['R', '11']},
 24: {';': ['R'], '=': ['R', '11'], 'relop': ['R', '11'], '+': ['R', '11']},
 26: {'end': ['R']},
 28: {')': ['R']},
 30: {'return': ['R']},
 32: {')': ['R']}}

In [58]:
print("\t\t\t\t\tACTION\t\t\t\t\t\t\t\tGOTO")
print('State', end=' ')
for t in terminals:
    print(t, end = '   ')
print('$', end=' ')
for t in non_terminals:
    print(t, end = ' ')
print("\n---------------------------------------------------------------------------------------------------------------------------------------------")
for i in range(0, len(action)):
    print(i, end = '     ')
    for t in terminals:
        try:
            print(action[i][t][0]+action[i][t][1], end = '   ')
        except:
            print('', end = ' ')
    try:
        print(action[i]['$'][0]+action[i]['$'][1], end = '   ')
    except:
        print('', end = '   ')
    for t in non_terminals:
        try:
            print(action[i][t][0]+action[i][t][1], end = '   ')
        except:
            print('', end = '   ')
    print('\n')

					ACTION								GOTO
State id   num   ;   (   )   =   +   while   return   exp   relop   $ TYPE CONTENT DOWHILE RETURN OPERATION EXP VAR FORMAT 
---------------------------------------------------------------------------------------------------------------------------------------------
0                                           

1                ac                           

2                                           

3     R11                                        

4     R11                                        

5                                           

6                                           

7                                           

8                                           

9                                           

10                                           

11                                           

12                                           

13                                           

14                                           

15 

In [62]:
from tabulate import tabulate
 
data = action
head = 'State id   num   ;   (   )   =   +   while   return   exp   relop   $ TYPE CONTENT DOWHILE RETURN OPERATION EXP VAR FORMAT '
headers = head.split()
 
table = tabulate(data, headers, tablefmt="grid")
 
print(table)

+------+----+----+------+----+----+---------+--------+---------+------+-------+-----+-----+-----+-----+-----+---------+----------+-------+---------+------+--------+-----------+-----------+----------+-------------+-------+--------+----------+
|      |    |    |      |    |    |         |        | State   | id   | num   | ;   | (   | )   | =   | +   | while   | return   | exp   | relop   | $    | TYPE   | CONTENT   | DOWHILE   | RETURN   | OPERATION   | EXP   | VAR    | FORMAT   |
+======+====+====+======+====+====+=========+========+=========+======+=======+=====+=====+=====+=====+=====+=========+==========+=======+=========+======+========+===========+===========+==========+=============+=======+========+==========+
| TYPE | (  | )  | TYPE | id | ;  | DOWHILE | return | id      | (    | while | num | (   | )   | id  | )   | relop   | id       | $     | main    | main | $      | end       | ;         | ;        | end         | )     | return | )        |
+------+----+----+------+----+--